# Compulsory assignment 1 - Using Dask-ml on large data
## Group #16
### Thomas Moen and Jørgen Kongsro

In [1]:
%matplotlib notebook


import numpy as np
import pandas as pd
import dask
import os


from operator import itemgetter

import numpy as np
import scipy

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import matplotlib.pyplot as plt
import skimage.io

from dask_ml.linear_model import LogisticRegression
from dask_ml.model_selection import train_test_split


In [ ]:
# Install Kaggle API
# How to setup: https://github.com/Kaggle/kaggle-api
# or visit: https://adityashrm21.github.io/Setting-Up-Kaggle/

#!pip install kaggle

In [ ]:
# Download Kaggle data using Kaggle API
#!kaggle competitions download -c dat300-ca1-autumn-2019


In [ ]:
# Unzip Kaggle data
#!unzip "dat300-ca1-autumn-2019.zip" -d "/tmp/whatever"

In [2]:
#choose here which imputation methods to use:

all_imputation_methods = ['col_means', 'correlated_columns']
imputation_methods = ['correlated_columns', 'col_means']



In [3]:
# List files in directory

if os.name == 'nt':
    workdir = 'C://Users//thomoe//Documents//myDAT300//dat300-ca1-autumn-2019//'
elif os.name == 'posix':
    workdir = '/Users/jorgenkongsro/Downloads/dat300-ca1-autumn-2019/'
    
os.listdir(workdir)


['y_test_sampleSubmission.csv', 'X_train.csv', 'y_train.csv', 'X_test.csv']

In [4]:
def import_data(csv_file):
    """ Import data from csv file

    :param data: a .csv separated dataset
    :return: a pandas data array, df
    
    """
    
    
    df = dd.read_csv(csv_file)
    return df

x_train_df = import_data(workdir + 'X_train.csv')
x_test_df = import_data(workdir + 'X_test.csv')
y_train_df = import_data(workdir + 'y_train.csv')



In [5]:
#get dimensions etc:

x_train_df_nrows = len(x_train_df)
x_train_df_ncols = len(x_train_df.columns)
x_test_df_nrows = len(x_test_df)
x_test_df_ncols = len(x_test_df.columns)

x_train_df_colnames = list(x_train_df.columns)



In [6]:
def percent_missing(dataframe):
    """ Check for percent missing values in dataframe
    :param data: dataframe
    :return: dataframe
    
    """
    missing_values = dataframe.isnull().sum()
    
    with ProgressBar():
        percent_missing = ((missing_values / dataframe.index.size) * 100).compute()
        
    return percent_missing


print(percent_missing(x_train_df))

"""
note: the results indicate that the features come in "tripets", e.g. f1 to f3 have quite similar missing%. 
We could impute some values very precisely by insert the mean of the other values within the triplet, 
if only one or two values are missing
"""

[########################################] | 100% Completed | 34.9s
f1      3.628002
f2      3.754723
f3      3.836795
f4      4.133142
f5      4.299061
f6      4.377457
f7      9.171993
f8      9.276405
f9      9.410288
f10     3.628002
f11     3.754723
f12     3.836795
f13     4.133142
f14     4.299061
f15     4.377457
f16     9.171993
f17     9.276405
f18     9.410288
f19     3.628002
f20     3.754723
f21     3.836795
f22     4.133142
f23     4.299061
f24     4.377457
f25     9.171993
f26     9.276405
f27     9.410288
f28     3.628002
f29     3.754723
f30     3.836795
          ...   
f133    9.171993
f134    9.276405
f135    9.410288
f136    3.628002
f137    3.754723
f138    3.836795
f139    4.133142
f140    4.299061
f141    4.377457
f142    9.171993
f143    9.276405
f144    9.410288
f145    3.628002
f146    3.754723
f147    3.836795
f148    4.133142
f149    4.299061
f150    4.377457
f151    9.171993
f152    9.276405
f153    9.410288
f154    3.628002
f155    3.754723
f156    3.8367

'\nnote: the results indicate that the features come in "tripets", e.g. f1 to f3 have quite similar missing%. \nWe could impute some values very precisely by insert the mean of the other values within the triplet, \nif only one or two values are missing\n'

In [15]:
def nan_filter(dataframe):
    """ Filter out NaN
    :param data: dataframe
    :return: dataframe
    
    """
    filtered = dataframe.dropna(how='all', axis= 0, subset=None, thresh=None).compute()

    return filtered

filtered = nan_filter(x_train_df)
percent_missing(filtered)

#THIS ONE DOES NOT WORK, THE AXIS PARAMETER SEEMS TO HAVE BEEN DEPRECATED. 
#DROPPING A WITHIN SPECIFYING ROWS OR COLUMNS DID NOT REMOVE ANY OF EITHER

#remove columns where all observations are NaN: 
#print(len(x_train_df.columns))
#print(len(x_train_df))
#x_train_df_nafiltered = x_train_df.dropna(how='all').compute()
#print(len(x_train_df.columns))
#print(len(x_train_df))



TypeError: dropna() got an unexpected keyword argument 'axis'

In [8]:
def drop_columns(df, thresh_perc, ncols = None, nrows = None):
    """ Drop columns which have more than 50% missing
    :param data: dataframe
    :return: dataframe
    """
    
    if ncols == None: ncols = len(df.columns)
    if nrows == None: nrows = len(df)
    
    nmissing = list(df.isna().sum().compute())
    colstodrop = [a for a in range(ncols) if nmissing[a]/nrows*100 >= thresh_perc]
    colstodrop = [df.columns[a] for a in colstodrop]
    return df.drop(colstodrop, axis='columns')
   


In [9]:
#delete columns (features) with more than N % missing observations from training data set: 
missing_perc = 50
x_train_df = drop_columns(x_train_df, missing_perc, x_train_df_ncols, x_train_df_nrows)

diff = x_train_df_ncols - len(x_train_df.columns)
if diff > 0: 
    print('Deleted', diff, 'features because they had more than', missing_perc, 'percent missing observations')
    x_train_df_ncols = len(x_train_df.columns)
else:
    print('No features had more than', missing_perc, 'percent missing observations')    

No features had more than 50 percent missing observations


In [105]:
#impute by inserting value from most closely correlated column: 

"""
I can't understand why this one doesn't work. The idea was to fill in NaN's from correlated columns.
The problem might be in the last line

"""

#will only correct if the correlation between columns is above this threshold: 
lowest_allowed_corr = 0.995

if 'correlated_columns' in imputation_methods:
    
    #if correlation matrix does not exist, first try to read it from file, if that does not work calculate it: 
    if dir().count('corrs') == 0: 
        try: 
            corrs = pd.read_csv(workdir + 'features_correlation_matrix.txt')
        except:
            corrs = x_train_df.corr('pearson')
    
    #impute for each feature feat:
    for feat in x_train_df_colnames:
        
        #order the feature names according to (absolute value of) correlations to feat:
        abscorr = [abs(a) for a in list(corrs[feat])]
        order = np.argsort(abscorr)[::-1]
        topfeatures = [x_train_df_colnames[a] for a in order]
        
        #remove features which are not sufficiently correlated to feat:
        mapper = dict(zip(x_train_df_colnames, abscorr))
        topfeatures = [a for a in topfeatures if mapper[a] >= lowest_allowed_corr]
        
        #correct using each feature in topfeatures, starting with the most strongly correlated otherfeature:
        for otherfeat in [a for a in topfeatures if a != feat]:
            print(feat, otherfeat)
            x_train_df_trim[feat] = x_train_df_trim[feat].fillna(x_train_df_trim[otherfeat]).compute()



f2 f3
f3 f2


In [23]:
#impute by inserting the mean of the column in question, for all columns: 
 
if 'col_means' in imputation_methods:
    
    #calculate mean (note that axis needs to be 0 to get columns, which is weird)
    miin = x_train_df.mean(axis = 0).compute()

    x_train_df_imean = x_train_df.fillna(dict(miin))


NotImplementedError: Series getitem in only supported for other series objects with matching partition structure

In [ ]:
def drop_rows(dataframe):
    """ Drop rows if more than 0 and less than 5% missing 
    :param data: dataframe
    :return: dataframe
    
    """
    rows_to_drop = list(percent_missing[(percent_missing > 0) & (percent_missing < 5)].index)
    data_clean = dataframe.dropna(subset=rows_to_drop)

In [ ]:
def impute_rows(dataframe,feature):
    """ Impute rows if more than 5% and less than 50% missing
    :param data: dataframe, feature
    :return: dataframe
    
    """
    value = median[feature]
    df_impute[feature] = df[feature].fillna(value)
    return df_impute

In [ ]:
# Print the size of a feature
print (x_train_df.count().compute())


In [12]:
# Build model and train

# Import libraries
import dask
import dask.array as da
from dask_ml.datasets import make_classification
from dask_ml.model_selection import train_test_split
from dask_ml.linear_model import LogisticRegression
from dask_ml.metrics import accuracy_score
from dask_ml.model_selection import IncrementalSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

import matplotlib.pyplot as plt

import numpy as np

In [21]:
X = x_train_df_imean
y = y_train_df

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y_train_df)


ValueError: Mismatched divisions. (None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None) != (None, None)